In [71]:
from kafka import KafkaProducer
import json
import json

import zlib
import base64
import json

import asyncio
import json

from signalr_async.net import Hub
from signalr_async.net.client import SignalRClient

In [72]:
def json_serializer(data):
    return json.dumps(data).encode("utf-8")

producer = KafkaProducer(bootstrap_servers=['192.168.86.103:9092'],
                         value_serializer=json_serializer)

def parse_data(text, zipped=True):
    """Parse json and jsonStream as returned by livetiming.formula1.com

    This function can only pass one data entry at a time, not a whole response.
    Timestamps and data need to be separated before and only the data must be passed as a string to be parsed.

    Args:
        text (str): The string which should be parsed
        zipped (bool): Whether or not the text is compressed. This is the case for '.z' data (e.g. position data=)

    Returns:
        Depending on data of which page is parsed:
            - a dictionary created as a result of loading json data
            - a string
    """
    if text[0] == '{':
        return json.loads(text)
    if text[0] == '"':
        text = text.strip('"')
    if zipped:
        text = zlib.decompress(base64.b64decode(text), -zlib.MAX_WBITS)
        return parse_data(text.decode('utf-8-sig'))
    print("Couldn't parse text")
    return text

In [73]:
class F1SignalRClient(SignalRClient):
    driver_list = {}
    timing_data = {}
    session_data = {}
    session_info = {}

    def SessionData(self, data):
        print('SessionData')
        print(data)
        producer.send("SessionData", data)


    def SessionInfo(self, data):
        print('SessionInfo')
        print(data)
        producer.send("SessionInfo", data)


    def DriverList(self, data):
        print('DriverList')
        print(data)
        for driver in data:
            producer.send("DriverList", data[driver])


    def TimingData(self, data):
        print('TimingData')
        print(data)
        for key in data['Lines']:
            producer.send("TimingData", {key: data['Lines'][key]})

    # def CarData(self, data):
    #     data = parse_data(data)
    #     print('test')
    #     producer.send("F1", data)
    #     print('test')
    #     # print('CarData')
    #     # merge(self.driver_list, drivers)

    def Position(self, data):
        print('Position')
        print(data)
        data = parse_data(data)
        producer.send("Position", data)


    async def _process_message(self, message):
        if hasattr(message, "result"):
            for k in message.result:
                if hasattr(self, k):
                    getattr(self, k)(message.result[k])
                elif hasattr(self, k[:-2]):
                    getattr(self, k[:-2])(message.result[k])
        elif hasattr(message, "arguments"):

            k, v, t = message.arguments
            if hasattr(self, k):
                    getattr(self, k)(v)
            elif hasattr(self, k[:-2]):
                getattr(self, k[:-2])(v)


_connection_url = 'https://livetiming.formula1.com/signalr'
hub = Hub("streaming")

async def run_client():
    async with F1SignalRClient(
            _connection_url,
            [hub],
            keepalive_interval=5,
    ) as client:
        await hub.invoke("Subscribe", ["Heartbeat", "CarData.z", "Position.z",
                       "ExtrapolatedClock", "TopThree", "RcmSeries",
                       "TimingStats", "TimingAppData",
                       "WeatherData", "TrackStatus", "DriverList",
                       "RaceControlMessages", "SessionInfo",
                       "SessionData", "LapCount", "TimingData"])
        await client.wait(timeout=5)



In [74]:
# https://github.com/iebb/PixooLiveTiming

await asyncio.gather(
        run_client()
    )



Position
zZY/b9swEMW/C2c7uDvy/lB75xaIhzZFB6PIYBRxiliZDH/3UtJR0MS6g1Ithg344XjvPf6ka/jyejn1p9dz6L5fw+H08nzpjy+/QxcIKO6B92QHTB1zh/ggOaomfQq78Oncv52eL6G7Bhw+Hvtj/15+hs/nw9vx56/yl6+h26sQ7sK34Rsm2oWn0DHE2y5QQyQq4CJOOIqS5iJKTVHMk8iy8DQJqYgQWipWmVXio3RQtbYSBp+VSXwrHFWtExYvqHoRceEFSmuW5eQnRE0LM9AaKsLoB4wwTUppcL3lhaL5pBKy+VY4qJpeJFAfpZBcNSbcjFjYfc9gtlTF1gkh+yxJOGWcDAZVsxlGnpZmywsHSZsZZ29uRsVFn2Kz7xx9L5U540GVmidMXPs0NmucJUXF3FQp1JDVU0YuKml5KClb7a7ywnltujFuM7lBsFBZ+54Yzx7avNfttvsba+gBY1lKbX3WcKyBIS0qtQZrqLoBxBtljTtoRPWm2JZYE2vjRWlt1kSrrDG4nzVaaQ0J1mYN175zvf8rsoZqyCx3s4ZB5u7KdlmTMJaSxA9gDc4PB135vWZ+KkPaKGus7kVOgK2xpjoogmuzhnyt6cbcyxqjyhpPa03W8PxKnj+QNfgPrJnfayT+J9b8uP0B
DriverList
{'1': {'RacingNumber': '1', 'BroadcastName': 'M VERSTAPPEN', 'FullName': 'Max VERSTAPPEN', 'Tla': 'VER', 'Line': 1, 'TeamName': 'Red Bull Racing', 'TeamColour': '3671C6', 'FirstName': 'Max', 'LastName': 'Verstappen', 'Reference': 'MAXVER01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER

CancelledError: 

## Data exploration

In [ ]:
timing = {'Lines': {'1': {'GapToLeader': 'LAP 61', 'IntervalToPositionAhead': {'Value': 'LAP 61', 'Catching': False}, 'Line': 1, 'Position': '1', 'ShowPosition': True, 'RacingNumber': '1', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.984', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.984'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.156'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.228'}], 'Speeds': {'I1': {'Value': '133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.604', 'Lap': 23}, 'LastLapTime': {'Value': '1:41.540', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 1}, '14': {'GapToLeader': '+16.302', 'IntervalToPositionAhead': {'Value': '+16.302', 'Catching': True}, 'Line': 2, 'Position': '2', 'ShowPosition': True, 'RacingNumber': '14', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.776', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '26.776'}, {'Stopped': False, 'Value': '47.743', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.743'}, {'Stopped': False, 'Value': '26.119', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.119'}], 'Speeds': {'I1': {'Value': '138', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '250', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '267', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.674', 'Lap': 43}, 'LastLapTime': {'Value': '1:40.638', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 2}, '31': {'GapToLeader': '+29.585', 'IntervalToPositionAhead': {'Value': '+12.703', 'Catching': False}, 'Line': 3, 'Position': '3', 'ShowPosition': True, 'RacingNumber': '31', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.474', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.474'}, {'Stopped': False, 'Value': '48.208', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.208'}, {'Stopped': False, 'Value': '26.845', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.845'}], 'Speeds': {'I1': {'Value': '125', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '131', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '247', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '252', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.528', 'Lap': 41}, 'LastLapTime': {'Value': '1:42.527', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 2}, '55': {'GapToLeader': '+46.529', 'IntervalToPositionAhead': {'Value': '+1.330', 'Catching': True}, 'Line': 8, 'Position': '8', 'ShowPosition': True, 'RacingNumber': '55', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.314', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.314'}, {'Stopped': False, 'Value': '47.757', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '47.757'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.971'}], 'Speeds': {'I1': {'Value': '125', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '269', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.449', 'Lap': 43}, 'LastLapTime': {'Value': '1:42.820', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '44': {'GapToLeader': '+31.628', 'IntervalToPositionAhead': {'Value': '+2.197', 'Catching': False}, 'Line': 4, 'Position': '4', 'ShowPosition': True, 'RacingNumber': '44', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.600', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.600'}, {'Stopped': False, 'Value': '48.261', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.261'}, {'Stopped': False, 'Value': '27.180', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.180'}], 'Speeds': {'I1': {'Value': '123', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '232', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '243', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:15.650', 'Lap': 33}, 'LastLapTime': {'Value': '1:43.041', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 2}, '16': {'GapToLeader': '+43.396', 'IntervalToPositionAhead': {'Value': '+5.596', 'Catching': True}, 'Line': 6, 'Position': '6', 'ShowPosition': True, 'RacingNumber': '16', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.831', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.831'}, {'Stopped': False, 'Value': '48.554', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.554'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '26.470'}], 'Speeds': {'I1': {'Value': '138', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '130', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '239', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:15.773', 'Lap': 46}, 'LastLapTime': {'Value': '1:42.305', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '10': {'GapToLeader': '+45.432', 'IntervalToPositionAhead': {'Value': '+2.036', 'Catching': False}, 'Line': 7, 'Position': '7', 'ShowPosition': True, 'RacingNumber': '10', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.013', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.013'}, {'Stopped': False, 'Value': '47.844', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '47.844'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '27.764'}], 'Speeds': {'I1': {'Value': '133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '135', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '259', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:15.831', 'Lap': 49}, 'LastLapTime': {'Value': '1:43.477', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '63': {'GapToLeader': '+37.889', 'IntervalToPositionAhead': {'Value': '+6.613', 'Catching': True}, 'Line': 5, 'Position': '5', 'ShowPosition': True, 'RacingNumber': '63', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.062', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.062'}, {'Stopped': False, 'Value': '48.313', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.313'}, {'Stopped': False, 'Value': '27.155', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.155'}], 'Speeds': {'I1': {'Value': '131', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '129', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '247', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '259', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.798', 'Lap': 48}, 'LastLapTime': {'Value': '1:43.530', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 1}, '22': {'GapToLeader': '+90.746', 'IntervalToPositionAhead': {'Value': '+45.340', 'Catching': False}, 'Line': 9, 'Position': '9', 'ShowPosition': True, 'RacingNumber': '22', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.018', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.018'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.912'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.572'}], 'Speeds': {'I1': {'Value': '120', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.680', 'Lap': 36}, 'LastLapTime': {'Value': '1:42.608', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '4': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+11.084', 'Catching': True}, 'Line': 10, 'Position': '10', 'ShowPosition': True, 'RacingNumber': '4', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.406', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.406'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.759'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.446'}], 'Speeds': {'I1': {'Value': '140', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.844', 'Lap': 46}, 'LastLapTime': {'Value': '1:41.216', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '81': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+2.029', 'Catching': False}, 'Line': 11, 'Position': '11', 'ShowPosition': True, 'RacingNumber': '81', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.216', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.216'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.366'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.163'}], 'Speeds': {'I1': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.513', 'Lap': 47}, 'LastLapTime': {'Value': '1:42.379', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '21': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+16.217', 'Catching': False}, 'Line': 13, 'Position': '13', 'ShowPosition': True, 'RacingNumber': '21', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.164', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '28.164'}, {'Stopped': False, 'Value': '49.132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '49.132'}, {'Stopped': False, 'Value': '27.089', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.089'}], 'Speeds': {'I1': {'Value': '122', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '130', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '248', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '266', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.561', 'Lap': 43}, 'LastLapTime': {'Value': '1:44.385', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '23': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+0.960', 'Catching': True}, 'Line': 15, 'Position': '15', 'ShowPosition': True, 'RacingNumber': '23', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.907', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.907'}, {'Stopped': False, 'Value': '48.838', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.838'}, {'Stopped': False, 'Value': '26.388', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.014'}], 'Speeds': {'I1': {'Value': '129', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '136', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '243', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '220', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.672', 'Lap': 24}, 'LastLapTime': {'Value': '1:43.133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '18': {'GapToLeader': '', 'IntervalToPositionAhead': {'Value': '', 'Catching': False}, 'Line': 20, 'Position': '20', 'ShowPosition': False, 'RacingNumber': '18', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': True, 'Status': 4, 'Sectors': [{'Stopped': False, 'Value': '24.037', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '24.037'}, {'Stopped': True, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2052}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.082'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '25.150'}], 'Speeds': {'I1': {'Value': '140', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.769', 'Lap': 18}, 'LastLapTime': {'Value': '1:36.321', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 53, 'NumberOfPitStops': 1}, '77': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+7.547', 'Catching': False}, 'Line': 12, 'Position': '12', 'ShowPosition': True, 'RacingNumber': '77', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.527', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.527'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.458'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.225'}], 'Speeds': {'I1': {'Value': '136', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.824', 'Lap': 19}, 'LastLapTime': {'Value': '1:42.622', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '2': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+28.437', 'Catching': False}, 'Line': 18, 'Position': '18', 'ShowPosition': True, 'RacingNumber': '2', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.647', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.647'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.845'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.803'}], 'Speeds': {'I1': {'Value': '116', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.302', 'Lap': 32}, 'LastLapTime': {'Value': '1:42.599', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 58, 'NumberOfPitStops': 3}, '20': {'GapToLeader': '2 L', 'IntervalToPositionAhead': {'Value': '+80.931', 'Catching': False}, 'Line': 19, 'Position': '19', 'ShowPosition': True, 'RacingNumber': '20', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '68.435', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '68.435'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '49.120'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.476'}], 'Speeds': {'I1': {'Value': '130', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '231', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:18.351', 'Lap': 41}, 'LastLapTime': {'Value': '1:48.018', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 57, 'NumberOfPitStops': 1}, '27': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+9.858', 'Catching': True}, 'Line': 16, 'Position': '16', 'ShowPosition': True, 'RacingNumber': '27', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.367', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.367'}, {'Stopped': False, 'Value': '47.690', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '47.690'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.797'}], 'Speeds': {'I1': {'Value': '129', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '131', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '259', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.991', 'Lap': 6}, 'LastLapTime': {'Value': '1:42.188', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfPitStops': 2, 'NumberOfLaps': 58}, '24': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+13.676', 'Catching': True}, 'Line': 14, 'Position': '14', 'ShowPosition': True, 'RacingNumber': '24', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.562', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.562'}, {'Stopped': False, 'Value': '48.727', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.727'}, {'Stopped': False, 'Value': '27.303', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.303'}], 'Speeds': {'I1': {'Value': '126', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '128', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '248', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '268', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.926', 'Lap': 5}, 'LastLapTime': {'Value': '1:43.592', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfPitStops': 2, 'NumberOfLaps': 59}, '11': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+12.346', 'Catching': True}, 'Line': 17, 'Position': '17', 'ShowPosition': True, 'RacingNumber': '11', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.084', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.084'}, {'Stopped': False, 'Value': '46.715', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '46.715'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.103'}], 'Speeds': {'I1': {'Value': '117', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '264', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.269', 'Lap': 5}, 'LastLapTime': {'Value': '1:45.989', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfPitStops': 4, 'NumberOfLaps': 58}}, 'Withheld': False, '_kf': True}


In [ ]:
for key in timing['Lines']:
    print({key: timing['Lines'][key]})

{'1': {'GapToLeader': 'LAP 61', 'IntervalToPositionAhead': {'Value': 'LAP 61', 'Catching': False}, 'Line': 1, 'Position': '1', 'ShowPosition': True, 'RacingNumber': '1', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.984', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.984'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.156'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'Pre